In [ ]:
# =============================================================================
# Setup and Configuration
# =============================================================================
# This notebook sets up all dependencies, API keys, and basic configurations
# Run each cell sequentially to ensure proper setup
# =============================================================================

# -----------------------------------------------------------------------------
# Section 1.1: Install Required Packages
# -----------------------------------------------------------------------------
# Install all necessary libraries for the travel planner
# This includes LangGraph, Google Generative AI, MCP, and utilities

INSTALL_INSTRUCTIONS = """
Run this in your terminal first:
pip install langgraph langchain-google-genai langchain-community
pip install streamlit requests beautifulsoup4 duckduckgo-search
pip install icalendar python-dotenv pandas
pip install mcp anthropic-mcp
"""


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.2: Import Core Libraries
# -----------------------------------------------------------------------------
# Import all necessary libraries for the project

import os
import json
import time
from datetime import datetime, timedelta
from typing import TypedDict, List, Dict, Any, Optional
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup

# LangChain and LangGraph imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Utility imports
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [ ]:
# --------------------------------------------
# Create .env file with Google API key
# Run this ONCE
# --------------------------------------------

with open(".env", "w") as f:
    f.write("GOOGLE_API_KEY=YOUR API KEY")

print("✅ .env file created with GOOGLE_API_KEY")


✅ .env file created with GOOGLE_API_KEY


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.3: Environment Setup
# -----------------------------------------------------------------------------
# Set up API keys and environment variables
# Create a .env file with your API keys

# Load environment variables from .env file
load_dotenv()

# Get API keys (you'll need to create a .env file with these)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Validate API keys
if not GOOGLE_API_KEY:
    print("⚠️ WARNING: GOOGLE_API_KEY not found in environment variables")
    print("Please create a .env file with: GOOGLE_API_KEY=your_key_here")
else:
    print("✅ Google API Key loaded successfully!")

✅ Google API Key loaded successfully!


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.4: Initialize Gemini Model
# -----------------------------------------------------------------------------
# Set up the Gemini model for all agents

def initialize_gemini(
    model_name: str = "gemini-2.0-flash-exp",
    temperature: float = 0.7,
    max_tokens: int = 8000
) -> ChatGoogleGenerativeAI:
    """
    Initialize the Gemini model with specified parameters.

    Args:
        model_name: Name of the Gemini model to use
        temperature: Controls randomness (0.0 = deterministic, 1.0 = creative)
        max_tokens: Maximum tokens in response

    Returns:
        Configured ChatGoogleGenerativeAI instance
    """
    if not GOOGLE_API_KEY:
        raise ValueError("Google API key not found. Please set GOOGLE_API_KEY in .env file")

    llm = ChatGoogleGenerativeAI(
        model=model_name,
        google_api_key=GOOGLE_API_KEY,
        temperature=temperature,
        max_tokens=max_tokens,
        timeout=120,  # 2 minute timeout
        max_retries=2
    )

    return llm

# Test the model initialization
try:
    test_llm = initialize_gemini()
    test_response = test_llm.invoke([HumanMessage(content="Say 'Setup successful!'")])
    print(f"✅ Gemini Model Test: {test_response.content}")
except Exception as e:
    print(f"❌ Error initializing Gemini: {str(e)}")


✅ Gemini Model Test: Setup successful!


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.5: Configuration Classes
# -----------------------------------------------------------------------------
# Define configuration structures for travel planning

class TravelConfig(TypedDict):
    """Configuration for travel planning parameters"""
    destination: str
    num_days: int
    travel_style: str  # Adventure, Relaxation, Culture, etc.
    budget_range: str  # Budget-Friendly, Mid-Range, Luxury
    start_date: datetime
    interests: List[str]
    headcount: int  # Number of travelers
    multi_city: bool  # Whether this is multi-city travel
    cities: List[str]  # List of cities if multi-city

class AgentTimeout:
    """Timeout configuration for different agents"""
    RESEARCH = 120  # 2 minutes
    WEATHER = 60    # 1 minute
    HOTEL = 120     # 2 minutes
    BUDGET = 90     # 1.5 minutes
    LOGISTICS = 120 # 2 minutes
    PLANNER = 180   # 3 minutes
    ACTIVITIES = 90 # 1.5 minutes

# Default configuration
DEFAULT_CONFIG = TravelConfig(
    destination="Paris, France",
    num_days=7,
    travel_style="Culture",
    budget_range="Mid-Range",
    start_date=datetime.today(),
    interests=["History & Culture", "Food & Dining"],
    headcount=2,
    multi_city=False,
    cities=[]
)

print("✅ Configuration classes defined!")
print(f"📋 Default Config: {DEFAULT_CONFIG['destination']}, {DEFAULT_CONFIG['num_days']} days")

✅ Configuration classes defined!
📋 Default Config: Paris, France, 7 days


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.6: Utility Functions
# -----------------------------------------------------------------------------
# Helper functions for logging, error handling, and data processing

def safe_api_call(func, timeout_seconds: int = 120, agent_name: str = "Agent"):
    """
    Wrapper for API calls with timeout protection.

    Args:
        func: Function to execute
        timeout_seconds: Maximum execution time
        agent_name: Name for logging

    Returns:
        Result of the function call

    Raises:
        TimeoutError: If execution exceeds timeout
    """
    import signal

    def timeout_handler(signum, frame):
        raise TimeoutError(f"{agent_name} exceeded {timeout_seconds}s timeout")

    # Set up timeout (Unix-based systems only)
    try:
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout_seconds)

        result = func()

        signal.alarm(0)  # Cancel the alarm
        return result
    except AttributeError:
        # Windows doesn't support SIGALRM, use threading instead
        from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeout

        with ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(func)
            try:
                return future.result(timeout=timeout_seconds)
            except FutureTimeout:
                raise TimeoutError(f"{agent_name} exceeded {timeout_seconds}s timeout")

def truncate_text(text: str, max_length: int = 2500) -> str:
    """
    Truncate text to avoid token limits.

    Args:
        text: Text to truncate
        max_length: Maximum character length

    Returns:
        Truncated text with indicator if cut
    """
    if not text or len(text) <= max_length:
        return text
    return text[:max_length] + "\n\n[Content truncated due to length...]"

def format_currency(amount: float, currency: str = "USD") -> str:
    """
    Format currency amounts consistently.

    Args:
        amount: Numeric amount
        currency: Currency code

    Returns:
        Formatted currency string
    """
    symbols = {"USD": "$", "EUR": "€", "GBP": "£", "JPY": "¥"}
    symbol = symbols.get(currency, currency)
    return f"{symbol}{amount:,.2f}"

def parse_date_range(start_date: datetime, num_days: int) -> List[datetime]:
    """
    Generate list of dates for trip.

    Args:
        start_date: Trip start date
        num_days: Number of days

    Returns:
        List of datetime objects
    """
    return [start_date + timedelta(days=i) for i in range(num_days)]

print("✅ Utility functions defined!")
print("🧪 Testing utility function:")
test_dates = parse_date_range(datetime.today(), 3)
print(f"   3-day trip dates: {[d.strftime('%Y-%m-%d') for d in test_dates]}")

✅ Utility functions defined!
🧪 Testing utility function:
   3-day trip dates: ['2026-01-03', '2026-01-04', '2026-01-05']


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.7: Logging Setup
# -----------------------------------------------------------------------------
# Set up structured logging for debugging and monitoring

import logging
from pathlib import Path

# Create logs directory if it doesn't exist
LOG_DIR = Path("logs")
LOG_DIR.mkdir(exist_ok=True)

def setup_logger(name: str = "travel_planner") -> logging.Logger:
    """
    Set up logger with file and console handlers.

    Args:
        name: Logger name

    Returns:
        Configured logger instance
    """
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    # Remove existing handlers
    logger.handlers.clear()

    # File handler
    log_file = LOG_DIR / f"{name}_{datetime.now().strftime('%Y%m%d')}.log"
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Formatter
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# Initialize logger
logger = setup_logger()
logger.info("🚀 Travel Planner initialized successfully!")
print(f"✅ Logger initialized! Logs saved to: {LOG_DIR}")


2026-01-03 12:18:50,529 - travel_planner - INFO - 🚀 Travel Planner initialized successfully!


✅ Logger initialized! Logs saved to: logs


In [ ]:
# -----------------------------------------------------------------------------
# Section 1.8: Verification Summary
# -----------------------------------------------------------------------------
# Final check of all setup components

print("\n" + "="*70)
print("SETUP VERIFICATION SUMMARY")
print("="*70)

checks = {
    "Google API Key": bool(GOOGLE_API_KEY),
    "Gemini Model": False,
    "Logging System": True,
    "Configuration": True,
    "Utilities": True
}

# Test Gemini connection
try:
    test_llm = initialize_gemini()
    checks["Gemini Model"] = True
except:
    pass

for component, status in checks.items():
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {component}")

all_ready = all(checks.values())
print("="*70)
if all_ready:
    print("🎉 All systems ready!")
else:
    print("⚠️ Some components need attention. Fix errors before proceeding.")
print("="*70)


SETUP VERIFICATION SUMMARY
✅ Google API Key
✅ Gemini Model
✅ Logging System
✅ Configuration
✅ Utilities
🎉 All systems ready!


In [ ]:
# =============================================================================
# Tools and MCP Server Integration
# =============================================================================
# This notebook creates custom tools and integrates MCP servers
# for web search, weather, and other external data sources
# =============================================================================

# -----------------------------------------------------------------------------
# Section 2.1: Import Dependencies
# -----------------------------------------------------------------------------
# Import necessary libraries for tools and MCP integration

from langchain_core.tools import tool
from typing import List, Dict, Any, Optional
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import json
from datetime import datetime, timedelta


try:
    logger
except NameError:
    from notebook_01_setup import logger, truncate_text

print("✅ Dependencies imported for tools and MCP")


✅ Dependencies imported for tools and MCP


In [ ]:
# Install ddgs into the SAME Python environment
import sys  # # lets us install into the kernel interpreter
!{sys.executable} -m pip install -U ddgs  # # installs/updates ddgs


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=25962) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.5/861.5 kB 2.3 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [ddgs]


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.2: Web Search Tool (DuckDuckGo)
# -----------------------------------------------------------------------------
# Clean setup:
# - Uses ddgs if installed
# - Falls back to duckduckgo_search if ddgs is not available
# - Returns a consistent list of {title, url, snippet}
# - Works as a LangChain tool (use .invoke for testing)

from langchain_core.tools import tool  # # LangChain tool decorator
from typing import List, Dict, Any  # # type hints for clean outputs


@tool
def web_search_tool(query: str, max_results: int = 6) -> List[Dict[str, Any]]:
    """
    Search the web using DuckDuckGo and return results.

    Args:
        query: Search query string
        max_results: Maximum number of results to return (default: 6)

    Returns:
        List of dictionaries containing title, url, and snippet
    """
    # # Import DDGS in a robust way so the notebook never crashes on import
    try:
        from ddgs import DDGS  # # new package name
    except Exception:
        from duckduckgo_search import DDGS  # # fallback to old package

    try:
        logger.info(f"🔍 Web search: {query}")  # # log query for debugging

        # # Run the search and force results into a list
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))  # # search results

        # # Normalize output fields to {title, url, snippet}
        formatted_results = []
        for r in results:
            formatted_results.append({
                "title": r.get("title", ""),
                "url": r.get("href", r.get("link", "")),
                "snippet": r.get("body", r.get("snippet", ""))
            })

        logger.info(f"✅ Found {len(formatted_results)} results")  # # log result count
        return formatted_results

    except Exception as e:
        logger.error(f"❌ Search error: {str(e)}")  # # log failures
        return [{
            "title": "Search Error",
            "url": "",
            "snippet": f"Error performing search: {str(e)}"
        }]


# -----------------------------------------------------------------------------
# Test the search tool (IMPORTANT: StructuredTool needs .invoke)
# -----------------------------------------------------------------------------
print("\n🧪 Testing Web Search Tool:")
test_results = web_search_tool.invoke({"query": "Paris travel tips", "max_results": 3})  # # correct call style
for i, result in enumerate(test_results, 1):
    print(f"{i}. {result['title'][:50]}...")


2026-01-03 12:29:33,978 - travel_planner - INFO - 🔍 Web search: Paris travel tips



🧪 Testing Web Search Tool:


2026-01-03 12:29:36,156 - travel_planner - INFO - ✅ Found 3 results


1. Find and save ideas about paris travel tips first ...
2. Paris Traveling Tips | TikTok...
3. 15 SAFETY TIPS for PARIS ! - YouTube...


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.3: Web Content Fetcher
# -----------------------------------------------------------------------------
# Tool to fetch and extract content from specific URLs

@tool
def fetch_webpage_content(url: str, timeout: int = 10) -> Dict[str, Any]:
    """
    Fetch and extract text content from a webpage.

    Args:
        url: URL to fetch
        timeout: Request timeout in seconds

    Returns:
        Dictionary with status, url, title, and text content

    Example:
        content = fetch_webpage_content("https://example.com")
    """
    logger.info(f"📄 Fetching: {url}")

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }

        response = requests.get(url, timeout=timeout, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()

        # Get text and clean it
        text = soup.get_text(separator=' ', strip=True)
        text = ' '.join(text.split())  # Normalize whitespace

        # Get title
        title = soup.title.string if soup.title else "No title"

        result = {
            "status": "success",
            "url": url,
            "title": title,
            "content": truncate_text(text, max_length=3000),
            "length": len(text)
        }

        logger.info(f"✅ Fetched {len(text)} characters")
        return result

    except Exception as e:
        logger.error(f"❌ Fetch error: {str(e)}")
        return {
            "status": "error",
            "url": url,
            "error": str(e),
            "content": ""
        }

# Test the fetcher
print("\n🧪 Testing Web Fetch Tool:")
print("(This may take a few seconds...)")


🧪 Testing Web Fetch Tool:
(This may take a few seconds...)


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.4: Weather Forecast Tool
# -----------------------------------------------------------------------------
# Free weather data using Open-Meteo API (no key required)

@tool
def get_weather_forecast(
    destination: str,
    start_date: str,
    num_days: int = 7
) -> Dict[str, Any]:
    """
    Get weather forecast for a destination using Open-Meteo API.

    Args:
        destination: City name or location
        start_date: Start date in YYYY-MM-DD format
        num_days: Number of days to forecast

    Returns:
        Dictionary with weather data including temperature, conditions, precipitation

    Example:
        weather = get_weather_forecast("Paris", "2024-06-01", 7)
    """
    logger.info(f"🌤️ Getting weather for {destination}")

    try:
        # Step 1: Geocode the destination
        geocode_url = "https://geocoding-api.open-meteo.com/v1/search"
        geocode_params = {
            "name": destination,
            "count": 1,
            "language": "en",
            "format": "json"
        }

        geo_response = requests.get(geocode_url, params=geocode_params, timeout=10)
        geo_response.raise_for_status()
        geo_data = geo_response.json()

        if not geo_data.get("results"):
            return {"error": f"Location not found: {destination}"}

        location = geo_data["results"][0]
        lat, lon = location["latitude"], location["longitude"]
        location_name = location.get("name", destination)

        # Step 2: Get weather forecast
        weather_url = "https://api.open-meteo.com/v1/forecast"

        # Parse start date
        start = datetime.strptime(start_date, '%Y-%m-%d')
        end = start + timedelta(days=num_days - 1)

        weather_params = {
            "latitude": lat,
            "longitude": lon,
            "daily": "temperature_2m_max,temperature_2m_min,weathercode,precipitation_probability_max,windspeed_10m_max",
            "timezone": "auto",
            "start_date": start.strftime('%Y-%m-%d'),
            "end_date": end.strftime('%Y-%m-%d')
        }

        weather_response = requests.get(weather_url, params=weather_params, timeout=10)
        weather_response.raise_for_status()
        weather_data = weather_response.json()

        # Step 3: Format the data
        daily = weather_data.get("daily", {})

        # Weather code descriptions
        weather_codes = {
            0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Foggy", 48: "Rime fog", 51: "Light drizzle", 53: "Moderate drizzle",
            55: "Dense drizzle", 61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
            80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
            95: "Thunderstorm", 96: "Thunderstorm with hail"
        }

        forecast = []
        dates = daily.get("time", [])
        temp_max = daily.get("temperature_2m_max", [])
        temp_min = daily.get("temperature_2m_min", [])
        codes = daily.get("weathercode", [])
        precip = daily.get("precipitation_probability_max", [])
        wind = daily.get("windspeed_10m_max", [])

        for i in range(min(len(dates), num_days)):
            code = codes[i] if i < len(codes) else 0
            forecast.append({
                "date": dates[i],
                "temp_max_c": temp_max[i] if i < len(temp_max) else None,
                "temp_min_c": temp_min[i] if i < len(temp_min) else None,
                "temp_max_f": round(temp_max[i] * 9/5 + 32, 1) if i < len(temp_max) else None,
                "temp_min_f": round(temp_min[i] * 9/5 + 32, 1) if i < len(temp_min) else None,
                "conditions": weather_codes.get(code, "Unknown"),
                "precipitation_prob": precip[i] if i < len(precip) else None,
                "wind_speed_kmh": wind[i] if i < len(wind) else None
            })

        result = {
            "location": f"{location_name} ({lat:.2f}°, {lon:.2f}°)",
            "forecast": forecast,
            "num_days": len(forecast)
        }

        logger.info(f"✅ Weather data retrieved for {num_days} days")
        return result

    except Exception as e:
        logger.error(f"❌ Weather error: {str(e)}")
        return {"error": str(e)}

# Test weather tool
print("\n🧪 Testing Weather Tool:")
# Call StructuredTool correctly using .invoke with named inputs
weather_test = get_weather_forecast.invoke({
    "destination": "Paris",  # required field name
    "start_date": datetime.today().strftime("%Y-%m-%d"),
    "days": 3
})

if "error" not in weather_test:
    print(f"📍 Location: {weather_test['location']}")
    print(f"📅 Days available: {weather_test['num_days']}")
    if weather_test['forecast']:
        first_day = weather_test['forecast'][0]
        print(f"🌡️ First day: {first_day['temp_max_c']}°C, {first_day['conditions']}")
else:
    print(f"⚠️ Weather test error: {weather_test.get('error')}")

2026-01-03 12:33:42,278 - travel_planner - INFO - 🌤️ Getting weather for Paris



🧪 Testing Weather Tool:


2026-01-03 12:33:43,869 - travel_planner - INFO - ✅ Weather data retrieved for 7 days


📍 Location: Paris (48.85°, 2.35°)
📅 Days available: 7
🌡️ First day: 4.0°C, Slight snow


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.5: Hotel Search Tool
# -----------------------------------------------------------------------------
# Search for hotel recommendations (simulated - can be enhanced with real APIs)

from langchain_core.tools import tool  # # tool decorator
from typing import Dict, Any  # # type hints


@tool
def search_hotels(
    destination: str,  # # city or location
    checkin_date: str,  # # YYYY-MM-DD
    checkout_date: str,  # # YYYY-MM-DD
    num_guests: int = 2,  # # number of guests
    budget_range: str = "Mid-Range"  # # Budget-Friendly, Mid-Range, Luxury
) -> Dict[str, Any]:
    """
    Search for hotels in a destination using web search results.
    Returns a simple list of recommendations + booking platform links.
    """
    logger.info(f"🏨 Searching hotels in {destination}")  # # log for debugging

    try:
        # # Build search query based on budget
        budget_terms = {
            "Budget-Friendly": "budget affordable cheap",
            "Mid-Range": "mid-range comfortable",
            "Luxury": "luxury boutique 5-star"
        }
        budget_keywords = budget_terms.get(budget_range, "")
        query = f"{destination} {budget_keywords} hotels best rated {num_guests} guests"  # # DDG query

        # # IMPORTANT FIX:
        # # web_search_tool is a LangChain StructuredTool, so call it with .invoke
        search_results = web_search_tool.invoke({  # # correct way to call a tool
            "query": query,
            "max_results": 5
        })

        # # Extract relevant information
        hotels = []
        for result in (search_results or [])[:5]:  # # defensive: handle None/empty
            hotels.append({
                "name": result.get("title", ""),  # # hotel name from title
                "description": (result.get("snippet", "") or "")[:200],  # # short snippet
                "booking_url": result.get("url", ""),  # # link
                "source": "Web Search"  # # attribution
            })

        # # Add booking platform links (direct searches)
        destination_plus = destination.replace(" ", "+")  # # URL-safe for query strings
        destination_dash = destination.replace(" ", "-")  # # URL-safe for Airbnb style

        booking_platforms = {
            "Booking.com": f"https://www.booking.com/searchresults.html?ss={destination_plus}",
            "Hotels.com": f"https://www.hotels.com/search.do?destination={destination_plus}",
            "Airbnb": f"https://www.airbnb.com/s/{destination_dash}/homes"
        }

        result = {
            "destination": destination,  # # echo back inputs
            "checkin_date": checkin_date,  # # echo back inputs
            "checkout_date": checkout_date,  # # echo back inputs
            "num_guests": num_guests,  # # echo back inputs
            "budget_range": budget_range,  # # echo back inputs
            "hotels": hotels,  # # recommendations
            "booking_platforms": booking_platforms  # # platform links
        }

        logger.info(f"✅ Found {len(hotels)} hotel recommendations")  # # success log
        return result

    except Exception as e:
        logger.error(f"❌ Hotel search error: {str(e)}")  # # error log
        return {"error": str(e)}  # # return error in-band


# -----------------------------------------------------------------------------
# Test hotel search (fixed invoke keys)
# -----------------------------------------------------------------------------
print("\n🧪 Testing Hotel Search Tool:")

hotel_test = search_hotels.invoke({  # # tool invoke
    "destination": "Paris",
    "checkin_date": "2024-06-01",
    "checkout_date": "2024-06-07",
    "num_guests": 2,
    "budget_range": "Mid-Range"
})

if isinstance(hotel_test, dict) and "error" not in hotel_test:
    print(f"🏨 Found {len(hotel_test.get('hotels', []))} hotels")  # # count hotels
    print(f"🔗 Booking platforms: {list(hotel_test.get('booking_platforms', {}).keys())}")  # # show platforms
else:
    print(f"⚠️ Hotel test error: {hotel_test.get('error') if isinstance(hotel_test, dict) else hotel_test}")  # # show error


2026-01-03 12:40:45,831 - travel_planner - INFO - 🏨 Searching hotels in Paris
2026-01-03 12:40:45,834 - travel_planner - INFO - 🔍 Web search: Paris mid-range comfortable hotels best rated 2 guests



🧪 Testing Hotel Search Tool:


2026-01-03 12:40:48,276 - travel_planner - INFO - ✅ Found 5 results
2026-01-03 12:40:48,278 - travel_planner - INFO - ✅ Found 5 hotel recommendations


🏨 Found 5 hotels
🔗 Booking platforms: ['Booking.com', 'Hotels.com', 'Airbnb']


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.6: Activity Booking Tool
# -----------------------------------------------------------------------------
# Find activity booking links for attractions

# What this includes:
# 1) Web search per activity (real-time)
# 2) Platform detection: Viator, GetYourGuide, TripAdvisor, Klook
# 3) Multiple booking options per activity (not just 1 link)
# 4) General platform links for the destination
# 5) Limit activities to avoid too many searches

from langchain_core.tools import tool  # # tool decorator for LangChain
from typing import Dict, Any, List  # # type hints


@tool
def find_activity_bookings(destination: str, activities: List[str], max_results: int = 3) -> Dict[str, Any]:
    """
    Find booking options for activities in a destination.

    Args:
        destination: City or location name
        activities: List of activity names (e.g., ["Eiffel Tower", "Seine River Cruise"])
        max_results: Max search results per activity (default 3)

    Returns:
        Dict with:
          - destination
          - activities: list of {activity, booking_options:[{platform,url,title,snippet}]}
          - platform_links: general links to major booking platforms for the destination
    """
    logger.info(f"🎫 Finding bookings for {len(activities)} activities")  # # log start

    try:
        # # Major booking platforms to detect from URLs
        platforms = ["Viator", "GetYourGuide", "TripAdvisor", "Klook"]  # # platform list

        activity_bookings: List[Dict[str, Any]] = []  # # final activity-level results

        # # Limit to avoid too many searches / rate limits
        for activity in activities[:5]:
            # # Build search query for each activity
            query = f"{activity} {destination} tickets booking"  # # query string

            # # FIX: web_search_tool is a StructuredTool -> must use .invoke
            results = web_search_tool.invoke({  # # correct tool invocation
                "query": query,
                "max_results": max_results
            })

            booking_options: List[Dict[str, Any]] = []  # # options for this activity

            # # Normalize each result and tag platform
            for result in (results or []):
                url = (result.get("url", "") or "")  # # url string
                url_lower = url.lower()  # # lowercase for matching

                # # Detect if major platforms are in URL
                platform_found = next(
                    (p for p in platforms if p.lower() in url_lower),
                    "Other"  # # default if none match
                )

                booking_options.append({
                    "platform": platform_found,  # # detected platform or Other
                    "url": url,  # # booking link
                    "title": result.get("title", ""),  # # result title
                    "snippet": (result.get("snippet", "") or "")[:200]  # # short snippet
                })

            activity_bookings.append({
                "activity": activity,  # # activity name
                "booking_options": booking_options  # # list of booking results
            })

        # # General platform links for the destination (kept from original idea)
        destination_dash = destination.replace(" ", "-").lower()  # # e.g., "new-york"
        destination_plus = destination.replace(" ", "+")  # # e.g., "New+York"

        platform_links = {
            "Viator": f"https://www.viator.com/{destination_dash}",
            "GetYourGuide": f"https://www.getyourguide.com/s/?q={destination_plus}",
            "TripAdvisor": f"https://www.tripadvisor.com/Search?q={destination_plus}",
            "Klook": f"https://www.klook.com/en-US/search/?query={destination_plus}",
        }

        result = {  # # final result payload
            "destination": destination,
            "activities": activity_bookings,
            "platform_links": platform_links
        }

        logger.info(f"✅ Found booking links for {len(activity_bookings)} activities")  # # success log
        return result

    except Exception as e:
        logger.error(f"❌ Activity booking error: {str(e)}")  # # error log
        return {"error": str(e)}  # # return error


# -----------------------------------------------------------------------------
# Test Activity Booking Tool
# -----------------------------------------------------------------------------
print("\n🧪 Testing Activity Booking Tool:")

activity_test = find_activity_bookings.invoke({  # # correct call for StructuredTool
    "destination": "Paris",
    "activities": ["Eiffel Tower", "Seine River Cruise"],
    "max_results": 3
})

if isinstance(activity_test, dict) and "error" not in activity_test:
    print(f"🎫 Found bookings for {len(activity_test.get('activities', []))} activities")



2026-01-03 12:54:33,248 - travel_planner - INFO - 🎫 Finding bookings for 2 activities
2026-01-03 12:54:33,250 - travel_planner - INFO - 🔍 Web search: Eiffel Tower Paris tickets booking



🧪 Testing Activity Booking Tool:


2026-01-03 12:54:34,358 - travel_planner - INFO - ✅ Found 3 results
2026-01-03 12:54:34,361 - travel_planner - INFO - 🔍 Web search: Seine River Cruise Paris tickets booking
2026-01-03 12:54:35,652 - travel_planner - INFO - ✅ Found 3 results
2026-01-03 12:54:35,654 - travel_planner - INFO - ✅ Found booking links for 2 activities


🎫 Found bookings for 2 activities


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.7: Multi-City Route Planner Tool
# -----------------------------------------------------------------------------
# Plan routes between multiple cities

from langchain_core.tools import tool  # # tool decorator for LangChain
from typing import Dict, Any, List  # # type hints
from datetime import datetime, timedelta  # # date utilities


@tool
def plan_multi_city_route(
    cities: List[str],  # # list of cities in visit order
    start_date: str,  # # YYYY-MM-DD
    total_days: int  # # total trip days
) -> Dict[str, Any]:
    """
    Plan a route through multiple cities, allocate days, and find transport options.
    """
    logger.info(f"🗺️ Planning multi-city route: {cities}")  # # log start

    try:
        num_cities = len(cities)  # # number of cities
        if num_cities < 2:  # # must have at least 2 cities
            return {"error": "Need at least 2 cities for multi-city planning"}

        # # Allocate days per city (simple even split with remainder given to earliest cities)
        days_per_city = total_days // num_cities  # # base allocation
        remaining_days = total_days % num_cities  # # extra days to distribute

        city_schedule = []  # # schedule output
        current_date = datetime.strptime(start_date, "%Y-%m-%d")  # # parse start date

        for i, city in enumerate(cities):
            # # Give extra days to the first few cities (remainder distribution)
            days_in_city = days_per_city + (1 if i < remaining_days else 0)  # # allocated days

            # # City end date based on days in city
            end_date = current_date + timedelta(days=days_in_city - 1)  # # departure date

            # # Add schedule entry
            city_schedule.append({
                "city": city,  # # city name
                "arrival": current_date.strftime("%Y-%m-%d"),  # # arrival date
                "departure": end_date.strftime("%Y-%m-%d"),  # # departure date
                "num_days": days_in_city,  # # number of days in city
                "order": i + 1  # # visit order
            })

            # # Move to next city: add 1 day for travel (kept from your original logic)
            current_date = end_date + timedelta(days=1)  # # next city's arrival date

        # # Search for transportation options between each city pair
        connections = []
        for i in range(len(cities) - 1):
            from_city = cities[i]  # # origin
            to_city = cities[i + 1]  # # destination

            query = f"{from_city} to {to_city} train flight bus"  # # transport query

            # # FIX: web_search_tool is a StructuredTool -> call with .invoke
            transport_results = web_search_tool.invoke({
                "query": query,
                "max_results": 2
            })

            # # Store BOTH title and url for better usability
            transport_options = []
            for r in (transport_results or []):
                transport_options.append({
                    "title": r.get("title", ""),  # # result title
                    "url": r.get("url", "")  # # result link
                })

            connections.append({
                "from": from_city,  # # from city
                "to": to_city,  # # to city
                "transport_options": transport_options  # # list of {title,url}
            })

        result = {
            "route": " → ".join(cities),  # # route string
            "total_days": total_days,  # # total trip days
            "num_cities": num_cities,  # # number of cities
            "city_schedule": city_schedule,  # # schedule list
            "connections": connections,  # # transport links
            "start_date": start_date  # # input start date
        }

        logger.info(f"✅ Multi-city route planned: {result['route']}")  # # success log
        return result

    except Exception as e:
        logger.error(f"❌ Route planning error: {str(e)}")  # # error log
        return {"error": str(e)}  # # error payload


# -----------------------------------------------------------------------------
# Test multi-city planner (use .invoke)
# -----------------------------------------------------------------------------
print("\n🧪 Testing Multi-City Route Planner:")

route_test = plan_multi_city_route.invoke({
    "cities": ["Paris", "Amsterdam", "Berlin"],
    "start_date": datetime.today().strftime("%Y-%m-%d"),
    "total_days": 10
})

if isinstance(route_test, dict) and "error" not in route_test:
    print(f"🗺️ Route: {route_test['route']}")
    print(f"🏙️ Cities: {route_test['num_cities']}, Days: {route_test['total_days']}")

    # -------------------------------------------------------------------------
    # Enhanced printing: show schedule + explicit travel days
    # -------------------------------------------------------------------------
    print("\n📅 City Schedule (with travel days):")

    schedule = route_test.get("city_schedule", [])  # # city schedule list
    conns = route_test.get("connections", [])  # # connections list

    for idx, s in enumerate(schedule):
        # # Print city schedule entry
        print(f"{s['order']}. {s['city']}: {s['arrival']} → {s['departure']} ({s['num_days']} days)")

        # # Print travel day between this city and next city (if any)
        if idx < len(conns):
            c = conns[idx]  # # matching connection
            next_city_arrival = schedule[idx + 1]["arrival"]  # # next city arrival date = travel day
            print(f"   Travel day: {c['from']} → {c['to']} on {next_city_arrival}")

    # -------------------------------------------------------------------------
    # Enhanced printing: show transport options with URLs
    # -------------------------------------------------------------------------
    print("\n🚆✈️ Connections (with links):")
    for c in conns:
        print(f"{c['from']} → {c['to']}:")
        for opt in c.get("transport_options", []):
            title = opt.get("title", "")  # # title
            url = opt.get("url", "")  # # url
            print(f"  - {title[:90]}")  # # preview title
            if url:
                print(f"    {url}")  # # show link
else:
    print(f"⚠️ Route test error: {route_test.get('error') if isinstance(route_test, dict) else route_test}")


2026-01-03 13:06:50,439 - travel_planner - INFO - 🗺️ Planning multi-city route: ['Paris', 'Amsterdam', 'Berlin']
2026-01-03 13:06:50,444 - travel_planner - INFO - 🔍 Web search: Paris to Amsterdam train flight bus



🧪 Testing Multi-City Route Planner:


2026-01-03 13:06:51,397 - travel_planner - INFO - ✅ Found 2 results
2026-01-03 13:06:51,400 - travel_planner - INFO - 🔍 Web search: Amsterdam to Berlin train flight bus
2026-01-03 13:06:53,032 - travel_planner - INFO - ✅ Found 2 results
2026-01-03 13:06:53,034 - travel_planner - INFO - ✅ Multi-city route planned: Paris → Amsterdam → Berlin


🗺️ Route: Paris → Amsterdam → Berlin
🏙️ Cities: 3, Days: 10

📅 City Schedule (with travel days):
1. Paris: 2026-01-03 → 2026-01-06 (4 days)
   Travel day: Paris → Amsterdam on 2026-01-07
2. Amsterdam: 2026-01-07 → 2026-01-09 (3 days)
   Travel day: Amsterdam → Berlin on 2026-01-10
3. Berlin: 2026-01-10 → 2026-01-12 (3 days)

🚆✈️ Connections (with links):
Paris → Amsterdam:
  - Paris to Amsterdam - 7 ways to travel via train, bus, rideshare, plane, and car
    https://www.rome2rio.com/s/Paris/Amsterdam
  - Paris to Amsterdam train | Eurostar tickets (Thalys)
    https://www.eurostar.com/us-en/train/paris-to-amsterdam
Amsterdam → Berlin:
  - How To Get From Amsterdam To Berlin : Train , Bus , Or Fly?
    https://www.hiddenholland.com/amsterdam-to-berlin/
  - Book, search & compare trains , buses , flights & ferries - Omio
    https://www.omio.com/


In [ ]:
# -----------------------------------------------------------------------------
# Section 2.8: Tool Collection and Summary
# -----------------------------------------------------------------------------
# Organize all tools for use in agents

# Create a dictionary of all available tools
AVAILABLE_TOOLS = {
    "web_search": web_search_tool,
    "fetch_webpage": fetch_webpage_content,
    "weather_forecast": get_weather_forecast,
    "search_hotels": search_hotels,
    "find_activities": find_activity_bookings,
    "plan_multi_city": plan_multi_city_route
}

# Tool descriptions for agent use
TOOL_DESCRIPTIONS = {
    "web_search": "Search the web for travel information, recommendations, and current data",
    "fetch_webpage": "Fetch full content from specific URLs for detailed information",
    "weather_forecast": "Get weather forecasts for destinations (free, no API key)",
    "search_hotels": "Find hotel recommendations and booking platform links",
    "find_activities": "Find booking links for activities and attractions",
    "plan_multi_city": "Plan optimal routes through multiple cities"
}

print("\n" + "="*70)
print("TOOLS SUMMARY")
print("="*70)
for tool_name, description in TOOL_DESCRIPTIONS.items():
    print(f"✅ {tool_name}: {description}")
print("="*70)
print(f"🎉 {len(AVAILABLE_TOOLS)} tools ready for use!")
print("📝 Proceed to define agents")
print("="*70)


TOOLS SUMMARY
✅ web_search: Search the web for travel information, recommendations, and current data
✅ fetch_webpage: Fetch full content from specific URLs for detailed information
✅ weather_forecast: Get weather forecasts for destinations (free, no API key)
✅ search_hotels: Find hotel recommendations and booking platform links
✅ find_activities: Find booking links for activities and attractions
✅ plan_multi_city: Plan optimal routes through multiple cities
🎉 6 tools ready for use!
📝 Proceed to define agents


In [ ]:
# =============================================================================
# Agent Definitions
# =============================================================================
# This notebook defines all specialized agents using Gemini and the tools
# Each agent has a specific role in the travel planning workflow
# =============================================================================

# -----------------------------------------------------------------------------
# Section 3.1: Import Dependencies and Setup
# -----------------------------------------------------------------------------

from typing import Dict, Any, List
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import time

# Import from previous notebooks
try:
    logger, AVAILABLE_TOOLS, initialize_gemini, AgentTimeout
except NameError:
    print("⚠️ Please run notebooks 01 and 02 first, or import required functions")

print("✅ Dependencies loaded for agent definitions")


✅ Dependencies loaded for agent definitions


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.2: Base Agent Class
# -----------------------------------------------------------------------------
# Create a base class with timeout protection and error handling

class TravelAgent:
    """
    Base class for all travel planning agents.
    Provides timeout protection, error handling, and logging.
    """

    def __init__(
        self,
        name: str,
        role: str,
        system_prompt: str,
        tools: List = None,
        timeout: int = 120,
        temperature: float = 0.7
    ):
        """
        Initialize a travel agent.

        Args:
            name: Agent name
            role: Agent role description
            system_prompt: System instructions for the agent
            tools: List of tools the agent can use
            timeout: Maximum execution time in seconds
            temperature: LLM temperature (0.0 = deterministic, 1.0 = creative)
        """
        self.name = name
        self.role = role
        self.system_prompt = system_prompt
        self.tools = tools or []
        self.timeout = timeout

        # Initialize Gemini model
        self.llm = initialize_gemini(temperature=temperature)

        logger.info(f"🤖 Agent '{name}' initialized")

    def invoke(self, messages: List, max_retries: int = 2) -> Dict[str, Any]:
        """
        Invoke the agent with timeout and retry logic.

        Args:
            messages: List of messages to send to agent
            max_retries: Maximum number of retry attempts

        Returns:
            Dictionary with response content and metadata
        """
        start_time = time.time()

        # Prepend system message
        full_messages = [SystemMessage(content=self.system_prompt)] + messages

        for attempt in range(max_retries + 1):
            try:
                logger.info(f"🚀 {self.name} starting (attempt {attempt + 1}/{max_retries + 1})")

                # Check timeout
                elapsed = time.time() - start_time
                if elapsed > self.timeout:
                    raise TimeoutError(f"{self.name} exceeded {self.timeout}s timeout")

                # Invoke LLM
                response = self.llm.invoke(full_messages)

                elapsed = time.time() - start_time
                logger.info(f"✅ {self.name} completed in {elapsed:.2f}s")

                return {
                    "agent": self.name,
                    "content": response.content,
                    "elapsed_time": elapsed,
                    "attempt": attempt + 1
                }

            except TimeoutError as e:
                logger.error(f"⏱️ {self.name} timeout: {str(e)}")
                raise

            except Exception as e:
                logger.warning(f"⚠️ {self.name} attempt {attempt + 1} failed: {str(e)}")

                if attempt == max_retries:
                    logger.error(f"❌ {self.name} failed after {max_retries + 1} attempts")
                    raise

                # Wait before retry
                time.sleep(2 ** attempt)  # Exponential backoff

        raise Exception(f"{self.name} failed unexpectedly")

print("✅ Base TravelAgent class defined")


✅ Base TravelAgent class defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.3: Research Agent
# -----------------------------------------------------------------------------
# Agent for researching destinations and gathering information

research_system_prompt = """You are an expert travel researcher specializing in finding
accurate, up-to-date information about destinations, attractions, and accommodations.

Your responsibilities:
- Search for relevant travel information using web searches
- Identify top-rated attractions, restaurants, and accommodations
- Find current prices and availability when possible
- Organize findings into clear categories
- Prioritize recent and highly-rated sources
- Keep responses concise and actionable

When researching:
1. Start with broad searches to understand the destination
2. Focus on official tourism sites, travel blogs, and review platforms
3. Extract specific details: names, locations, prices, ratings
4. Avoid lengthy descriptions - focus on key facts
5. Cite sources when important

Output Format:
Organize your research into sections:
- **Attractions**: Top sights and activities
- **Dining**: Recommended restaurants with cuisines and price ranges
- **Accommodations**: Hotel options by budget range
- **Local Tips**: Transportation, safety, customs
- **Sources**: Key URLs for booking and more info

Be concise. Focus on quality over quantity (3-5 recommendations per category)."""

research_agent = TravelAgent(
    name="ResearchAgent",
    role="Destination Research Specialist",
    system_prompt=research_system_prompt,
    tools=[AVAILABLE_TOOLS["web_search"], AVAILABLE_TOOLS["fetch_webpage"]],
    timeout=AgentTimeout.RESEARCH,
    temperature=0.6  # Slightly lower for factual accuracy
)

print("✅ Research Agent defined")


2026-01-03 13:13:20,515 - travel_planner - INFO - 🤖 Agent 'ResearchAgent' initialized


✅ Research Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.4: Weather Agent
# -----------------------------------------------------------------------------
# Agent for analyzing weather and providing packing recommendations

weather_system_prompt = """You are a weather analysis and travel preparation expert.

Your responsibilities:
- Analyze weather forecasts for travel destinations
- Provide day-by-day weather summaries
- Suggest appropriate clothing and packing items
- Recommend weather-appropriate activities
- Warn about extreme weather conditions
- Give practical preparation tips

When analyzing weather:
1. Summarize conditions clearly (temperature, precipitation, wind)
2. Highlight any weather concerns (extreme heat/cold, rain, storms)
3. Suggest clothing layers and essential items
4. Recommend indoor alternatives for bad weather days
5. Consider activities suitable for each day's forecast

Output Format:
**Weather Overview**
- General climate conditions for the trip period
- Any notable weather patterns or concerns

**Daily Forecast**
Day 1 (Date): Conditions, Temp, Precip probability
Day 2 (Date): ...

**Packing Recommendations**
- Essential clothing items
- Weather-specific gear
- Health/safety items

**Activity Suggestions**
- Best days for outdoor activities
- Indoor alternatives if needed

Keep it practical and concise."""

weather_agent = TravelAgent(
    name="WeatherAgent",
    role="Weather Analysis & Packing Specialist",
    system_prompt=weather_system_prompt,
    tools=[AVAILABLE_TOOLS["weather_forecast"]],
    timeout=AgentTimeout.WEATHER,
    temperature=0.5  # Lower for consistent weather interpretation
)

print("✅ Weather Agent defined")


2026-01-03 13:14:03,781 - travel_planner - INFO - 🤖 Agent 'WeatherAgent' initialized


✅ Weather Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.5: Hotel Agent
# -----------------------------------------------------------------------------
# Agent for finding and recommending accommodations

hotel_system_prompt = """You are an accommodation specialist with expertise in finding
the best hotels, hostels, and vacation rentals for different budgets and preferences.

Your responsibilities:
- Search for accommodations matching traveler preferences
- Compare options across different booking platforms
- Consider location, amenities, reviews, and value
- Provide booking links and price estimates
- Suggest neighborhoods to stay in

When recommending hotels:
1. Match recommendations to the stated budget range
2. Consider location relative to main attractions
3. Highlight key amenities (WiFi, breakfast, parking, etc.)
4. Note review scores and recent feedback
5. Provide both specific hotels and general booking platform links

Budget Ranges:
- Budget-Friendly: Hostels, budget hotels, Airbnb rooms ($30-80/night)
- Mid-Range: 3-4 star hotels, nice Airbnbs ($80-200/night)
- Luxury: 4-5 star hotels, luxury properties ($200+/night)

Output Format:
**Recommended Accommodations**
1. [Hotel Name] - [Neighborhood]
   - Price range: $X-Y per night
   - Highlights: [Key amenities/features]
   - Booking: [URL or platform]

**Best Neighborhoods**
- [Neighborhood 1]: Why it's good for travelers
- [Neighborhood 2]: ...

**Booking Platforms**
- Booking.com: [URL]
- Hotels.com: [URL]
- Airbnb: [URL]

Focus on 3-5 top recommendations."""

hotel_agent = TravelAgent(
    name="HotelAgent",
    role="Accommodation Specialist",
    system_prompt=hotel_system_prompt,
    tools=[AVAILABLE_TOOLS["search_hotels"], AVAILABLE_TOOLS["web_search"]],
    timeout=AgentTimeout.HOTEL,
    temperature=0.6
)

print("✅ Hotel Agent defined")

2026-01-03 13:14:18,458 - travel_planner - INFO - 🤖 Agent 'HotelAgent' initialized


✅ Hotel Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.6: Budget Estimator Agent
# -----------------------------------------------------------------------------
# Agent for calculating trip costs and creating budgets

budget_system_prompt = """You are a travel budget expert specializing in cost estimation
and financial planning for trips.

Your responsibilities:
- Calculate realistic trip costs based on travel style and destination
- Break down expenses by category (accommodation, food, transport, activities)
- Provide daily budget estimates
- Suggest money-saving tips
- Account for hidden costs and contingencies

Budget Categories:
1. **Accommodation**: Hotels, hostels, Airbnb
2. **Food & Dining**: Meals, snacks, drinks
3. **Transportation**: Local transport, intercity travel
4. **Activities & Attractions**: Entry fees, tours, experiences
5. **Shopping & Misc**: Souvenirs, personal items
6. **Contingency**: 10-15% buffer for unexpected costs

Budget Ranges by Style:
- Budget-Friendly: $50-100/day per person
- Mid-Range: $100-250/day per person
- Luxury: $250-500+/day per person

Output Format:
**Total Trip Budget Estimate**
Total: $X,XXX for Y people, Z days

**Daily Budget Breakdown** (per person)
- Accommodation: $XX
- Food: $XX (Breakfast $X, Lunch $X, Dinner $X)
- Local Transport: $XX
- Activities: $XX
- Miscellaneous: $XX
Daily Total: $XXX

**Multi-Day Budget**
Total for Z days: $X,XXX

**Money-Saving Tips**
1. [Specific tip for this destination]
2. ...

**Payment Tips**
- Best credit cards for travel
- ATM/cash recommendations
- Tipping customs

Be realistic with estimates. Cite sources when using specific price data."""

budget_agent = TravelAgent(
    name="BudgetAgent",
    role="Budget & Financial Planning Specialist",
    system_prompt=budget_system_prompt,
    tools=[AVAILABLE_TOOLS["web_search"]],
    timeout=AgentTimeout.BUDGET,
    temperature=0.5  # Lower for consistent financial calculations
)

print("✅ Budget Agent defined")

2026-01-03 13:14:35,880 - travel_planner - INFO - 🤖 Agent 'BudgetAgent' initialized


✅ Budget Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.7: Logistics Agent
# -----------------------------------------------------------------------------
# Agent for planning transportation and routes

logistics_system_prompt = """You are a logistics and transportation expert specializing
in efficient route planning and travel connections.

Your responsibilities:
- Plan optimal routes between attractions and cities
- Suggest appropriate transportation modes
- Estimate realistic travel times
- Provide public transport guidance
- Recommend walking routes when feasible
- Plan efficient daily itineraries to minimize transit time

Transportation Options:
- **Walking**: For nearby locations (<1.5km)
- **Public Transit**: Buses, metro, trams (most cost-effective)
- **Rideshare/Taxi**: For convenience or late hours
- **Rental Car**: For multiple destinations or remote areas
- **Intercity**: Trains, buses, flights between cities

When planning routes:
1. Group nearby attractions together
2. Consider operating hours and peak times
3. Account for meal breaks and rest
4. Suggest morning/afternoon/evening activities
5. Provide specific transit options (metro lines, bus numbers)
6. Estimate total travel time per day

Output Format:
**Transportation Overview**
- Best way to get around: [Public transit/Walking/Rental car]
- Transit passes available: [Day pass, multi-day, tourist card]
- Approximate costs: $X per day

**Daily Route Plans**
Day 1:
- Morning: [Location A] → [Location B] (15 min walk)
- Afternoon: [Location B] → [Location C] (Metro Line 2, 20 min)
- Evening: [Location C] → [Location D] (Taxi, 10 min)

**Intercity Connections** (if multi-city)
- [City A] → [City B]: [Train 2hr 30min / Flight 1hr] + recommendations

**Transit Tips**
- [Specific advice for this destination]

Focus on efficiency and realism."""

logistics_agent = TravelAgent(
    name="LogisticsAgent",
    role="Transportation & Route Planning Specialist",
    system_prompt=logistics_system_prompt,
    tools=[AVAILABLE_TOOLS["web_search"], AVAILABLE_TOOLS["plan_multi_city"]],
    timeout=AgentTimeout.LOGISTICS,
    temperature=0.6
)

print("✅ Logistics Agent defined")

2026-01-03 13:14:56,493 - travel_planner - INFO - 🤖 Agent 'LogisticsAgent' initialized


✅ Logistics Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.8: Itinerary Planner Agent
# -----------------------------------------------------------------------------
# Master agent that creates the final detailed itinerary

planner_system_prompt = """You are a senior travel planner specializing in creating
detailed, personalized day-by-day itineraries.

Your responsibilities:
- Synthesize research from other agents into cohesive itineraries
- Create realistic daily schedules with time slots
- Balance activities with rest and meals
- Match activities to traveler interests and style
- Consider weather, logistics, and opening hours
- Provide practical tips and recommendations

Itinerary Structure:
**Day X: [Theme/Focus]** (Date)

**Morning (9:00 AM - 12:00 PM)**
- [Activity/Attraction]
  - What: [Brief description]
  - Where: [Location/address]
  - Duration: [Time needed]
  - Cost: $X per person
  - Tips: [Booking info, best time to visit]

**Lunch (12:00 PM - 1:30 PM)**
- [Restaurant recommendation]
  - Cuisine: [Type]
  - Price: $$ (estimate)
  - Location: [Near what]

**Afternoon (1:30 PM - 5:00 PM)**
[Similar format]

**Evening (5:00 PM - 9:00 PM)**
[Similar format]

**Day Notes**
- Weather: [Conditions from weather agent]
- Transportation: [How to get around]
- Budget: $XXX estimated for the day

Key Principles:
1. **Realistic pacing**: Don't overpack days (2-3 major activities max)
2. **Proximity**: Group nearby attractions
3. **Timing**: Consider opening hours and crowd patterns
4. **Flexibility**: Build in buffer time
5. **Variety**: Mix activity types (museums, outdoors, food, culture)
6. **Weather-aware**: Adjust activities based on forecast

For multi-city trips:
- Allocate days appropriately per city
- Include travel days with lighter activities
- Note intercity transportation details

Be specific with names, locations, and actionable details. This itinerary
should be ready to follow without additional research."""

planner_agent = TravelAgent(
    name="PlannerAgent",
    role="Master Itinerary Planner",
    system_prompt=planner_system_prompt,
    tools=[],  # Planner synthesizes info, doesn't need tools
    timeout=AgentTimeout.PLANNER,
    temperature=0.7  # Balanced creativity and structure
)

print("✅ Planner Agent defined")

2026-01-03 13:15:12,573 - travel_planner - INFO - 🤖 Agent 'PlannerAgent' initialized


✅ Planner Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.9: Activities & Booking Agent
# -----------------------------------------------------------------------------
# Agent for finding booking links for activities

activities_system_prompt = """You are an activities and booking specialist helping
travelers find and book experiences.

Your responsibilities:
- Extract specific activities/attractions from itineraries
- Find official booking platforms and links
- Recommend tour operators and experience providers
- Provide price estimates for activities
- Suggest skip-the-line or combo tickets

Major Booking Platforms:
- **Viator**: Wide selection, good reviews
- **GetYourGuide**: Popular tours and attractions
- **TripAdvisor Experiences**: Verified reviews
- **Klook**: Strong in Asia-Pacific
- **Official venue websites**: Often cheapest

When finding bookings:
1. Identify 5-8 key activities from the itinerary
2. Search for each on major booking platforms
3. Include direct official websites when available
4. Note if advance booking is required/recommended
5. Mention skip-the-line options
6. Provide price estimates

Output Format:
**Recommended Bookings**

**1. [Activity/Attraction Name]**
- Official Website: [URL]
- Book via Viator: [URL]
- Book via GetYourGuide: [URL]
- Price: $XX-XX per person
- Tips: [Book advance? Skip-the-line available?]

**2. [Next Activity]**
[Same format]

**General Booking Platforms**
- Viator: [Destination page URL]
- GetYourGuide: [Destination page URL]
- TripAdvisor: [Destination page URL]

**Booking Tips**
- [Specific advice for this destination]
- When to book in advance
- Cancellation policies to look for

Focus on the most popular/must-do activities."""

activities_agent = TravelAgent(
    name="ActivitiesAgent",
    role="Activity & Booking Specialist",
    system_prompt=activities_system_prompt,
    tools=[AVAILABLE_TOOLS["find_activities"], AVAILABLE_TOOLS["web_search"]],
    timeout=AgentTimeout.ACTIVITIES,
    temperature=0.6
)

print("✅ Activities Agent defined")


2026-01-03 13:15:27,098 - travel_planner - INFO - 🤖 Agent 'ActivitiesAgent' initialized


✅ Activities Agent defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 3.10: Agent Registry and Testing
# -----------------------------------------------------------------------------
# Organize all agents and test basic functionality

# Create agent registry
AGENT_REGISTRY = {
    "research": research_agent,
    "weather": weather_agent,
    "hotel": hotel_agent,
    "budget": budget_agent,
    "logistics": logistics_agent,
    "planner": planner_agent,
    "activities": activities_agent
}

# Test each agent with a simple query
def test_agent(agent: TravelAgent, test_query: str) -> bool:
    """Test an agent with a simple query."""
    try:
        messages = [HumanMessage(content=test_query)]
        response = agent.invoke(messages)

        print(f"✅ {agent.name}: OK ({response['elapsed_time']:.2f}s)")
        return True
    except Exception as e:
        print(f"❌ {agent.name}: FAILED - {str(e)}")
        return False

print("\n" + "="*70)
print("AGENT TESTING")
print("="*70)

# Simple test queries for each agent
test_queries = {
    "research": "Give me 2 top attractions in Paris",
    "weather": "What's important to know about weather in Paris in June?",
    "hotel": "Suggest 1 mid-range hotel in Paris",
    "budget": "Estimate daily budget for mid-range Paris trip",
    "logistics": "How to get around Paris efficiently?",
    "planner": "Create a simple 1-day Paris itinerary outline",
    "activities": "Find booking options for Eiffel Tower"
}

results = {}
for agent_key, agent in AGENT_REGISTRY.items():
    test_query = test_queries.get(agent_key, "Hello, can you help with travel planning?")
    results[agent_key] = test_agent(agent, test_query)
    time.sleep(2)  # Rate limiting

print("="*70)
success_count = sum(results.values())
total_count = len(results)
print(f"✅ {success_count}/{total_count} agents passed basic tests")

if success_count == total_count:
    print("🎉 All agents ready! Proceed to Notebook 04 for LangGraph workflow")
else:
    print("⚠️ Some agents failed. Check errors above.")
print("="*70)

2026-01-03 13:15:40,576 - travel_planner - INFO - 🚀 ResearchAgent starting (attempt 1/3)



AGENT TESTING


2026-01-03 13:15:41,715 - travel_planner - INFO - ✅ ResearchAgent completed in 1.14s


✅ ResearchAgent: OK (1.14s)


2026-01-03 13:15:43,720 - travel_planner - INFO - 🚀 WeatherAgent starting (attempt 1/3)
2026-01-03 13:15:50,224 - travel_planner - INFO - ✅ WeatherAgent completed in 6.50s


...

✅ LogisticsAgent: OK (9.58s)


2026-01-03 13:16:13,517 - travel_planner - INFO - 🚀 PlannerAgent starting (attempt 1/3)
2026-01-03 13:16:18,451 - travel_planner - INFO - ✅ PlannerAgent completed in 4.93s


✅ PlannerAgent: OK (4.93s)


2026-01-03 13:16:20,458 - travel_planner - INFO - 🚀 ActivitiesAgent starting (attempt 1/3)
2026-01-03 13:16:28,395 - travel_planner - INFO - ✅ ActivitiesAgent completed in 7.94s


✅ ActivitiesAgent: OK (7.94s)
✅ 7/7 agents passed basic tests
🎉 All agents ready! Proceed to Notebook 04 for LangGraph workflow


In [ ]:
# =============================================================================
# LangGraph Workflow
# =============================================================================
# This notebook builds the LangGraph workflow that orchestrates all agents
# Creates a stateful graph for the complete travel planning process
# =============================================================================

# -----------------------------------------------------------------------------
# Section 4.1: Import Dependencies
# -----------------------------------------------------------------------------

from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
import operator
from datetime import datetime

# Import from previous notebooks
try:
    logger, AGENT_REGISTRY, TravelConfig
except NameError:
    print("⚠️ Please run notebooks 01-03 first")

print("✅ Dependencies imported for LangGraph workflow")


✅ Dependencies imported for LangGraph workflow


In [ ]:
# -----------------------------------------------------------------------------
# Section 4.2: Define Workflow State
# -----------------------------------------------------------------------------
# The state tracks all information as it flows through the workflow

from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
import operator
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

class TravelPlannerState(TypedDict):
    """
    Updated State with revision tracking for cyclic graphs.
    """
    # User input
    destination: str
    num_days: int
    travel_style: str
    budget_range: str
    start_date: str
    interests: list
    headcount: int
    multi_city: bool
    cities: list

    # Agent outputs
    research_results: str
    weather_analysis: str
    hotel_recommendations: str
    budget_estimate: str
    logistics_plan: str
    final_itinerary: str
    activity_bookings: str

    # Workflow control
    messages: Annotated[Sequence[BaseMessage], operator.add]
    current_step: str
    revision_count: int  # NEW: Tracks how many times we've looped back

    # Metadata
    workflow_start_time: float
    workflow_end_time: float
    total_cost_estimate: float

print("✅ TravelPlannerState defined (with cyclic capabilities)")

✅ TravelPlannerState defined (with cyclic capabilities)


In [ ]:
import re  # # used for budget parsing and other simple extraction
import json  # # used for budget JSON parsing

# -----------------------------------------------------------------------------
# Section 4.3: Define Node Functions (Cyclic Workflow Version)
# -----------------------------------------------------------------------------

def research_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Research node: Gather destination information.
    Uses ResearchAgent to search for attractions, restaurants, accommodations.
    """
    logger.info("🔍 Starting research node")

    # Initialize revision count for the cyclic workflow if not present
    if state.get("revision_count") is None:
        state["revision_count"] = 0

    try:
        # Build research prompt
        research_prompt = f"""
        Research {state['destination']} for a {state['num_days']}-day trip.

        Trip Details:
        - Travel Style: {state['travel_style']}
        - Budget: {state['budget_range']}
        - Start Date: {state['start_date']}
        - Travelers: {state['headcount']} people
        - Interests: {', '.join(state['interests']) if state['interests'] else 'General'}

        Find:
        1. Top attractions and activities
        2. Highly-rated restaurants (various price points)
        3. Recommended accommodations
        4. Local transportation options
        5. Cultural tips and local customs

        Focus on recent, high-quality sources. Keep concise.
        """

        # Invoke research agent
        agent = AGENT_REGISTRY["research"]
        response = agent.invoke([HumanMessage(content=research_prompt)])

        # Update state
        state["research_results"] = response["content"]
        state["current_step"] = "research_complete"
        state["messages"].append(AIMessage(
            content=f"✅ Research completed for {state['destination']}"
        ))

        logger.info("✅ Research node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Research node failed: {str(e)}")
        state["errors"].append(f"Research: {str(e)}")
        state["research_results"] = f"Research failed: {str(e)}"
        return state


def weather_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Weather node: Get forecast and packing recommendations.
    Uses WeatherAgent to analyze weather and suggest preparations.
    """
    logger.info("🌤️ Starting weather node")

    try:
        weather_prompt = f"""
        Analyze weather for {state['destination']} from {state['start_date']}
        for {state['num_days']} days.

        Use the weather forecast tool to get actual data, then provide:
        1. Daily weather summary
        2. Packing list recommendations
        3. Weather-appropriate activity suggestions
        4. Any weather warnings or concerns

        Consider the travel style: {state['travel_style']}
        """

        agent = AGENT_REGISTRY["weather"]
        response = agent.invoke([HumanMessage(content=weather_prompt)])

        state["weather_analysis"] = response["content"]
        state["current_step"] = "weather_complete"
        state["messages"].append(AIMessage(
            content="✅ Weather forecast and packing tips ready"
        ))

        logger.info("✅ Weather node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Weather node failed: {str(e)}")
        state["errors"].append(f"Weather: {str(e)}")
        state["weather_analysis"] = "Weather data unavailable"
        return state


def hotel_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Hotel node: Find accommodation recommendations.
    Uses HotelAgent to search for suitable hotels/accommodations.
    Includes logic to handle retries if the Planner rejects previous results.
    """
    logger.info("🏨 Starting hotel node")

    try:
        # Calculate checkout date
        from datetime import datetime, timedelta
        start = datetime.strptime(state['start_date'], '%Y-%m-%d')
        checkout = (start + timedelta(days=state['num_days'])).strftime('%Y-%m-%d')

        # Check if this is a retry loop
        is_retry = state.get("revision_count", 0) > 0
        retry_instruction = ""
        if is_retry:
            logger.info("🔄 Hotel node executing retry logic (broadening search)")
            retry_instruction = """
            IMPORTANT: Previous search results were insufficient.
            Please BROADEN your search. Look for any available accommodations,
            even if they slightly deviate from the budget or style,
            but strictly prioritize availability and rating.
            """

        hotel_prompt = f"""
        Find accommodations for {state['destination']}.

        Requirements:
        - Check-in: {state['start_date']}
        - Check-out: {checkout}
        - Guests: {state['headcount']}
        - Budget: {state['budget_range']}
        - Style: {state['travel_style']}

        {retry_instruction}

        Use the hotel search tool and web search to find:
        1. 3-5 specific hotel recommendations
        2. Best neighborhoods to stay
        3. Booking platform links
        4. Price estimates per night

        Consider location relative to main attractions.
        """

        agent = AGENT_REGISTRY["hotel"]
        response = agent.invoke([HumanMessage(content=hotel_prompt)])

        state["hotel_recommendations"] = response["content"]
        state["current_step"] = "hotel_complete"
        state["messages"].append(AIMessage(
            content="✅ Hotel recommendations ready" + (" (Retry)" if is_retry else "")
        ))

        logger.info("✅ Hotel node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Hotel node failed: {str(e)}")
        state["errors"].append(f"Hotel: {str(e)}")
        state["hotel_recommendations"] = "Hotel search unavailable"
        return state


def budget_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Budget node: Calculate trip cost estimates.
    Uses BudgetAgent to create comprehensive budget breakdown.
    """
    logger.info("💰 Starting budget node")

    try:
        budget_prompt = f"""
Estimate trip budget for {state['destination']}.

Trip Details:
- Duration: {state['num_days']} days
- Travelers: {state['headcount']} people
- Budget Range: {state['budget_range']}
- Travel Style: {state['travel_style']}

Using the research results below for price context:
{state.get('research_results', 'No research data')[:1000]}
{state.get('hotel_recommendations', 'No hotel data')[:1000]}

Provide:

A) Assumptions (must be explicit numbers):
- lodging_per_night_total (for the whole group)
- lodging_nights = num_days - 1
- food_per_person_per_day
- local_transport_per_person_per_day
- activities_per_person_per_day
- misc_pct (as a decimal like 0.10)

B) Math (show the actual calculations):
- lodging_total = lodging_per_night_total * lodging_nights
- food_total = food_per_person_per_day * headcount * num_days
- local_transport_total = local_transport_per_person_per_day * headcount * num_days
- activities_total = activities_per_person_per_day * headcount * num_days
- subtotal = lodging_total + food_total + local_transport_total + activities_total
- misc_total = subtotal * misc_pct
- total_trip_cost = subtotal + misc_total
- per_person_total = total_trip_cost / headcount
- daily_total_all_people = total_trip_cost / num_days

C) Output:
1) A short bullet summary of totals
2) A daily per-person breakdown (lodging, food, transport, activities, misc)
3) Money-saving tips
4) Payment/currency advice

Be realistic for {state['budget_range']} and {state['travel_style']}.
Use USD.

Output format requirement:
Include a JSON block between these markers so we can parse totals:

JSON_START
{{"currency":"USD","headcount":{state['headcount']},"num_days":{state['num_days']},
"total_trip_cost":0,
"per_person_total":0,
"daily_total_all_people":0,
"breakdown":{{"lodging":0,"food":0,"local_transport":0,"activities":0,"misc":0}}}}
JSON_END

Rules:
- total_trip_cost must be the total for ALL travelers for the whole trip.
- Numeric fields must be numbers (no $ signs, no commas).
"""

        agent = AGENT_REGISTRY["budget"]
        response = agent.invoke([HumanMessage(content=budget_prompt)])

        state["budget_estimate"] = response["content"]
        state["current_step"] = "budget_complete"

        # -------------------------------------------------------------
        # Try to extract total cost (improved parsing: JSON-first, fallback)
        # -------------------------------------------------------------
        state["total_cost_estimate"] = 0.0

        json_match = re.search(
            r"JSON_START\s*(\{.*?\})\s*JSON_END",
            response["content"],
            re.S
        )

        if json_match:
            try:
                budget_json = json.loads(json_match.group(1))
                state["total_cost_estimate"] = float(budget_json.get("total_trip_cost", 0.0))
                state["budget_structured"] = budget_json
            except Exception:
                pass

        # Fallback: first $ amount if JSON missing or invalid
        if state["total_cost_estimate"] == 0.0:
            cost_match = re.search(r"\$[\d,]+", response["content"])
            if cost_match:
                cost_str = cost_match.group().replace("$", "").replace(",", "")
                try:
                    state["total_cost_estimate"] = float(cost_str)
                except Exception:
                    state["total_cost_estimate"] = 0.0

        state["messages"].append(AIMessage(
            content="✅ Budget estimate completed"
        ))

        logger.info("✅ Budget node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Budget node failed: {str(e)}")
        state["errors"].append(f"Budget: {str(e)}")
        state["budget_estimate"] = "Budget calculation unavailable"
        return state


def logistics_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Logistics node: Plan transportation and routes.
    Uses LogisticsAgent to optimize travel routes and suggest transport.
    """
    logger.info("🗺️ Starting logistics node")

    try:
        # Check if multi-city
        if state["multi_city"] and state["cities"]:
            logistics_prompt = f"""
            Plan logistics for multi-city trip: {' → '.join(state['cities'])}

            Trip Details:
            - Total Duration: {state['num_days']} days
            - Travelers: {state['headcount']} people
            - Start Date: {state['start_date']}

            Use the multi-city planning tool to:
            1. Allocate days per city
            2. Find transportation between cities (train/flight/bus)
            3. Estimate travel times and costs
            4. Suggest arrival/departure times

            Then for each city, suggest:
            - Local transportation options
            - How to get around efficiently
            - Transit passes available

            Research results for context:
            {state.get('research_results', '')[:1000]}
            """
        else:
            logistics_prompt = f"""
            Plan local logistics for {state['destination']}.

            Based on research results:
            {state.get('research_results', '')[:1500]}

            Provide:
            1. Best way to get around (metro, bus, walking, taxi, rental car)
            2. Transit passes and costs
            3. Approximate daily transportation budget
            4. Tips for efficient routing between attractions
            5. Airport/station transfer options

            Consider {state['num_days']} days and {state['travel_style']} style.
            """

        agent = AGENT_REGISTRY["logistics"]
        response = agent.invoke([HumanMessage(content=logistics_prompt)])

        state["logistics_plan"] = response["content"]
        state["current_step"] = "logistics_complete"
        state["messages"].append(AIMessage(
            content="✅ Logistics plan ready"
        ))

        logger.info("✅ Logistics node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Logistics node failed: {str(e)}")
        state["errors"].append(f"Logistics: {str(e)}")
        state["logistics_plan"] = "Logistics planning unavailable"
        return state


def planner_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Planner node: Create final detailed itinerary.
    Master agent that synthesizes all information into day-by-day plan.
    ACTS AS SUPERVISOR: Checks if critical info (like Hotels) is missing and requests revision.
    """
    logger.info("📋 Starting planner node (Evaluation Phase)")

    # 1. Evaluate Data Quality
    hotels_content = state.get("hotel_recommendations", "").lower()
    revision_count = state.get("revision_count", 0)
    max_retries = 1  # How many times we allow looping back

    # Check if hotels are missing or unavailable
    hotel_missing = "unavailable" in hotels_content or len(hotels_content) < 100

    # 2. DECISION: Loop back or Proceed?
    if hotel_missing and revision_count < max_retries:
        logger.warning(f"⚠️ Planner detected missing hotel data. Requesting revision (Attempt {revision_count + 1})")

        # Signal the router to loop back
        state["final_itinerary"] = "REVISE_HOTEL"
        state["revision_count"] = revision_count + 1

        state["messages"].append(AIMessage(
            content="⚠️ Hotel data missing or insufficient. Looping back to Hotel Agent for a broader search..."
        ))
        return state

    # 3. Proceed with generating the itinerary
    try:
        planner_prompt = f"""
        Create a detailed {state['num_days']}-day itinerary for {state['destination']}.

        Trip Details:
        - Destination: {state['destination']}
        - Duration: {state['num_days']} days
        - Start Date: {state['start_date']}
        - Travelers: {state['headcount']} people
        - Travel Style: {state['travel_style']}
        - Budget: {state['budget_range']}
        - Interests: {', '.join(state['interests']) if state['interests'] else 'General'}

        Information from other agents:

        RESEARCH:
        {state.get('research_results', 'Not available')[:2000]}

        WEATHER:
        {state.get('weather_analysis', 'Not available')[:1000]}

        HOTELS:
        {state.get('hotel_recommendations', 'Not available')[:1000]}

        BUDGET:
        {state.get('budget_estimate', 'Not available')[:800]}

        LOGISTICS:
        {state.get('logistics_plan', 'Not available')[:1000]}

        Create a comprehensive day-by-day itinerary with:
        - Time-slotted activities (Morning, Afternoon, Evening)
        - Specific attraction/restaurant names
        - Estimated costs and duration
        - Transportation between locations
        - Meal recommendations
        - Practical tips

        Format clearly by day. Be specific and actionable.
        Make it realistic - don't overpack days!
        """

        agent = AGENT_REGISTRY["planner"]
        response = agent.invoke([HumanMessage(content=planner_prompt)])

        state["final_itinerary"] = response["content"]
        state["current_step"] = "planner_complete"
        state["messages"].append(AIMessage(
            content="✅ Complete itinerary created"
        ))

        logger.info("✅ Planner node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Planner node failed: {str(e)}")
        state["errors"].append(f"Planner: {str(e)}")
        state["final_itinerary"] = f"Itinerary creation failed: {str(e)}"
        return state


def activities_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Activities node: Find booking links for activities.
    Uses ActivitiesAgent to find booking platforms for key activities.
    """
    logger.info("🎫 Starting activities node")

    # If the workflow is in revision mode, skip this step (though the graph should route around it)
    if state.get("final_itinerary") == "REVISE_HOTEL":
        return state

    try:
        activities_prompt = f"""
        Find booking links for activities in this itinerary:

        {state.get('final_itinerary', '')[:2500]}

        Extract 5-8 key activities/attractions and find:
        1. Official website booking links
        2. Major booking platform links (Viator, GetYourGuide, TripAdvisor, Klook)
        3. Price estimates
        4. Booking tips (advance required? skip-the-line?)

        Destination: {state['destination']}

        Focus on must-do activities and popular attractions.
        """

        agent = AGENT_REGISTRY["activities"]
        response = agent.invoke([HumanMessage(content=activities_prompt)])

        state["activity_bookings"] = response["content"]
        state["current_step"] = "activities_complete"
        state["messages"].append(AIMessage(
            content="✅ Activity booking links found"
        ))

        logger.info("✅ Activities node completed")
        return state

    except Exception as e:
        logger.error(f"❌ Activities node failed: {str(e)}")
        state["errors"].append(f"Activities: {str(e)}")
        state["activity_bookings"] = "Activity booking search unavailable"
        return state


def finalize_node(state: TravelPlannerState) -> TravelPlannerState:
    """
    Finalize node: Wrap up workflow and prepare final output.
    """
    logger.info("✨ Finalizing workflow")

    import time
    state["workflow_end_time"] = time.time()

    total_time = state["workflow_end_time"] - state.get("workflow_start_time", 0)

    state["messages"].append(AIMessage(
        content=f"""
🎉 Travel plan complete!
⏱️ Total processing time: {total_time:.1f}s
📍 Destination: {state['destination']}
📅 Duration: {state['num_days']} days
👥 Travelers: {state['headcount']}
💰 Estimated cost: ${state.get('total_cost_estimate', 0):,.2f}
        """
    ))

    state["current_step"] = "complete"
    logger.info("✅ Workflow finalized")

    return state


print("✅ All node functions defined")

✅ All node functions defined


In [ ]:
# -----------------------------------------------------------------------------
# Section 4.4: Build the LangGraph Workflow
# -----------------------------------------------------------------------------
# Create the graph and connect all nodes

# =============================================================================
# REVISED SECTION 4.4: Graph Construction with Conditional Edges (The Loop)
# =============================================================================

def router_check(state: TravelPlannerState) -> str:
    """
    This is the 'traffic cop' of the workflow.
    It reads the state from the Planner and decides where to go next.
    """
    if state.get("final_itinerary") == "REVISE_HOTEL":
        return "hotel"      # Path to go BACKWARDS
    return "activities"     # Path to go FORWARDS

def create_travel_planner_workflow():
    # Initialize the graph
    workflow = StateGraph(TravelPlannerState)

    # 1. Add all the nodes we defined in Section 4.3
    workflow.add_node("research", research_node)
    workflow.add_node("weather", weather_node)
    workflow.add_node("hotel", hotel_node)
    workflow.add_node("budget", budget_node)
    workflow.add_node("logistics", logistics_node)
    workflow.add_node("planner", planner_node)
    workflow.add_node("activities", activities_node)
    workflow.add_node("finalize", finalize_node)

    # 2. Define the LINEAR path
    workflow.set_entry_point("research")
    workflow.add_edge("research", "weather")
    workflow.add_edge("weather", "hotel")
    workflow.add_edge("hotel", "budget")
    workflow.add_edge("budget", "logistics")
    workflow.add_edge("logistics", "planner")

    # 3. Define the CYCLIC path (The "Smart" part)
    # This tells the graph: "When you finish the Planner, run router_check to see what to do."
    workflow.add_conditional_edges(
        "planner",
        router_check,
        {
            "hotel": "hotel",           # Map the router result 'hotel' to the 'hotel' node
            "activities": "activities"  # Map the router result 'activities' to the 'activities' node
        }
    )

    # 4. Final steps
    workflow.add_edge("activities", "finalize")
    workflow.add_edge("finalize", END)

    # Compile with memory (for state tracking)
    from langgraph.checkpoint.memory import MemorySaver
    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

# Re-initialize the workflow variable
travel_workflow = create_travel_planner_workflow()

print("🔄 SUCCESS: The workflow is now officially cyclic!")
print("The Planner can now send the Hotel Agent back to work if results are poor.")


🔄 SUCCESS: The workflow is now officially cyclic!
The Planner can now send the Hotel Agent back to work if results are poor.


In [ ]:
# -----------------------------------------------------------------------------
# Section 4.5: Helper Function to Run Workflow
# -----------------------------------------------------------------------------

def plan_trip(
    destination: str,
    num_days: int = 7,
    travel_style: str = "Culture",
    budget_range: str = "Mid-Range",
    start_date: str = None,
    interests: list = None,
    headcount: int = 2,
    multi_city: bool = False,
    cities: list = None
) -> dict:
    """
    Execute the complete travel planning workflow.

    Args:
        destination: Primary destination
        num_days: Trip duration
        travel_style: Adventure, Relaxation, Culture, Family-Friendly, Luxury
        budget_range: Budget-Friendly, Mid-Range, Luxury
        start_date: Trip start date (YYYY-MM-DD) - defaults to today
        interests: List of interests
        headcount: Number of travelers
        multi_city: Whether this is a multi-city trip
        cities: List of cities for multi-city trips

    Returns:
        Complete state dictionary with all results
    """
    import time

    # Set defaults
    if start_date is None:
        start_date = datetime.today().strftime('%Y-%m-%d')
    if interests is None:
        interests = []
    if cities is None:
        cities = [destination] if not multi_city else []

    # Initialize state
    initial_state = {
        "destination": destination,
        "num_days": num_days,
        "travel_style": travel_style,
        "budget_range": budget_range,
        "start_date": start_date,
        "interests": interests,
        "headcount": headcount,
        "multi_city": multi_city,
        "cities": cities,
        "research_results": "",
        "weather_analysis": "",
        "hotel_recommendations": "",
        "budget_estimate": "",
        "logistics_plan": "",
        "final_itinerary": "",
        "activity_bookings": "",
        "messages": [],
        "current_step": "initialized",
        "errors": [],
        "workflow_start_time": time.time(),
        "workflow_end_time": 0.0,
        "total_cost_estimate": 0.0
    }

    logger.info(f"🚀 Starting travel planning workflow for {destination}")

    # Configure for the workflow
    config = {"configurable": {"thread_id": f"trip_{int(time.time())}"}}

    # Run the workflow
    final_state = None
    for output in travel_workflow.stream(initial_state, config):
        # Stream outputs as they're generated
        for node_name, node_output in output.items():
            if node_name != "__end__":
                print(f"📍 Completed: {node_name}")
                final_state = node_output

    return final_state

print("✅ Helper function 'plan_trip' ready")


✅ Helper function 'plan_trip' ready


In [ ]:
# -----------------------------------------------------------------------------
# Section 4.6: Test the Workflow
# -----------------------------------------------------------------------------
# Run a simple test to verify the workflow works

print("\n" + "="*70)
print("WORKFLOW TEST")
print("="*70)
print("Running a 3-day Paris trip test...")
print("This will take 2-5 minutes depending on API speeds.")
print("="*70 + "\n")

# Run test (comment out if you want to skip)
test_result = plan_trip(
    destination="Paris, France",
    num_days=3,
    travel_style="Culture",
    budget_range="Mid-Range",
    interests=["Food & Dining", "History & Culture"],
    headcount=2
)

print("\n" + "="*70)
print("WORKFLOW TEST RESULTS")
print("="*70)
if test_result:
    print(f"✅ Workflow completed: {test_result['current_step']}")
    print(f"⏱️ Processing time: {test_result['workflow_end_time'] - test_result['workflow_start_time']:.1f}s")
    print(f"💰 Estimated cost: ${test_result.get('total_cost_estimate', 0):,.2f}")
    print(f"⚠️ Errors: {len(test_result.get('errors', []))}")

    if test_result.get('final_itinerary'):
        print("\n📋 Itinerary preview (first 500 chars):")
        print(test_result['final_itinerary'][:500] + "...")
else:
    print("❌ Workflow test failed")

print("="*70)
print("🎉 Workflow ready!")
print("="*70)

2026-01-03 16:40:37,864 - travel_planner - INFO - 🚀 Starting travel planning workflow for Paris, France
2026-01-03 16:40:37,882 - travel_planner - INFO - 🔍 Starting research node
2026-01-03 16:40:37,886 - travel_planner - INFO - 🚀 ResearchAgent starting (attempt 1/3)



WORKFLOW TEST
Running a 3-day Paris trip test...
This will take 2-5 minutes depending on API speeds.



2026-01-03 16:40:44,249 - travel_planner - INFO - ✅ ResearchAgent completed in 6.36s
2026-01-03 16:40:44,251 - travel_planner - INFO - ✅ Research node completed
2026-01-03 16:40:44,254 - travel_planner - INFO - 🌤️ Starting weather node
2026-01-03 16:40:44,254 - travel_planner - INFO - 🚀 WeatherAgent starting (attempt 1/3)


📍 Completed: research


2026-01-03 16:40:45,442 - travel_planner - INFO - ✅ WeatherAgent completed in 1.19s
2026-01-03 16:40:45,445 - travel_planner - INFO - ✅ Weather node completed
2026-01-03 16:40:45,451 - travel_planner - INFO - 🏨 Starting hotel node
2026-01-03 16:40:45,453 - travel_planner - INFO - 🚀 HotelAgent starting (attempt 1/3)


...

📍 Completed: logistics


2026-01-03 16:41:21,728 - travel_planner - INFO - ✅ PlannerAgent completed in 15.16s
2026-01-03 16:41:21,736 - travel_planner - INFO - ✅ Planner node completed
2026-01-03 16:41:21,754 - travel_planner - INFO - 🎫 Starting activities node
2026-01-03 16:41:21,754 - travel_planner - INFO - 🚀 ActivitiesAgent starting (attempt 1/3)


📍 Completed: planner


2026-01-03 16:41:27,247 - travel_planner - INFO - ✅ ActivitiesAgent completed in 5.49s
2026-01-03 16:41:27,249 - travel_planner - INFO - ✅ Activities node completed
2026-01-03 16:41:27,251 - travel_planner - INFO - ✨ Finalizing workflow
2026-01-03 16:41:27,255 - travel_planner - INFO - ✅ Workflow finalized


📍 Completed: activities
📍 Completed: finalize

WORKFLOW TEST RESULTS
✅ Workflow completed: complete
⏱️ Processing time: 49.4s
💰 Estimated cost: $1,400.00
⚠️ Errors: 0

📋 Itinerary preview (first 500 chars):
Okay, here is a detailed 3-day itinerary for Paris, France, designed for two travelers interested in food, dining, history, and culture, with a mid-range budget, starting January 3, 2026. The itinerary considers weather conditions, logistics, and provides practical tips for a smooth experience.

**Day 1: Iconic Paris & Latin Quarter Charm** (2026-01-03)

**Morning (9:00 AM - 12:00 PM)**
- Eiffel Tower
  - What: Visit the iconic Eiffel Tower. Take the elevator to the top for panoramic views of Pa...
🎉 Workflow ready!


In [ ]:
# =============================================================================
# SECTION 5: RUN & TEST THE CYCLIC TRAVEL PLANNER
# =============================================================================
import time

def run_travel_planner(requirements: dict):
    """
    Initializes the state and executes the cyclic LangGraph workflow.
    """
    print(f"🚀 Initializing Trip to {requirements['destination']}...")

    # Prepare the initial state
    # This matches the TravelPlannerState schema
    initial_state = {
        "destination": requirements.get("destination"),
        "num_days": requirements.get("num_days", 3),
        "travel_style": requirements.get("travel_style", "Balanced"),
        "budget_range": requirements.get("budget_range", "Moderate"),
        "start_date": requirements.get("start_date", "2024-06-01"),
        "interests": requirements.get("interests", ["Sightseeing"]),
        "headcount": requirements.get("headcount", 2),
        "multi_city": requirements.get("multi_city", False),
        "cities": requirements.get("cities", []),
        "messages": [HumanMessage(content=f"Plan a trip to {requirements['destination']}")],
        "revision_count": 0,
        "workflow_start_time": time.time(),
        "errors": []
    }

    # Configuration for the checkpointer (Memory)
    config = {"configurable": {"thread_id": "test_trip_001"}}

    # Execute the Graph
    print("🧠 The Agents are now collaborating (this may take 1-2 minutes)...")
    try:
        final_output = travel_workflow.invoke(initial_state, config)
        return final_output
    except Exception as e:
        print(f"❌ Critical Workflow Error: {e}")
        return None

# -----------------------------------------------------------------------------
# INPUT YOUR REQUIREMENTS HERE
# -----------------------------------------------------------------------------
my_trip_requirements = {
    "destination": "Italy (Rome & Florence)", # Main region
    "cities": ["Rome", "Florence"],           # Specific cities
    "multi_city": True,                       # Enable multi-city logic
    "num_days": 5,                            # Number of nights/days
    "start_date": "2024-09-15",               # Start date
    "headcount": 3,                           # Number of people
    "travel_style": "Luxury & Art focused",    # Kind of style
    "budget_range": "$5000 - $7000 total",    # Your budget
    "interests": ["Renaissance Art", "Local Food", "Walking Tours", "Wine Tasting"]
}

# -----------------------------------------------------------------------------
# EXECUTION AND DISPLAY
# -----------------------------------------------------------------------------
result = run_travel_planner(my_trip_requirements)

if result:
    print("\n" + "="*80)
    print("✨ TRIP PLAN COMPLETED SUCCESSFULLY ✨")
    print("="*80)

    # Check if a loop happened
    if result.get("revision_count", 0) > 0:
        print(f"🔄 NOTE: The workflow self-corrected {result['revision_count']} time(s) to improve hotel data.")

    print(f"\n💰 FINAL BUDGET ESTIMATE: ${result.get('total_cost_estimate', 0):,.2f}")

    print("\n📅 ITINERARY SUMMARY:")
    print("-" * 20)
    print(result.get("final_itinerary"))

    print("\n🎫 BOOKING LINKS & ACTIVITIES:")
    print("-" * 20)
    print(result.get("activity_bookings"))

    print("\n🌤️ WEATHER ADVISORY:")
    print("-" * 20)
    print(result.get("weather_analysis"))

    print("\n" + "="*80)
else:
    print("❌ Failed to generate the plan. Check your API Key and Logs.")

2026-01-03 16:43:03,808 - travel_planner - INFO - 🔍 Starting research node
2026-01-03 16:43:03,808 - travel_planner - INFO - 🚀 ResearchAgent starting (attempt 1/3)


🚀 Initializing Trip to Italy (Rome & Florence)...
🧠 The Agents are now collaborating (this may take 1-2 minutes)...


2026-01-03 16:43:12,329 - travel_planner - INFO - ✅ ResearchAgent completed in 8.52s
2026-01-03 16:43:12,335 - travel_planner - INFO - ✅ Research node completed
2026-01-03 16:43:12,341 - travel_planner - INFO - 🌤️ Starting weather node
2026-01-03 16:43:12,342 - travel_planner - INFO - 🚀 WeatherAgent starting (attempt 1/3)
2026-01-03 16:43:13,745 - travel_planner - INFO - ✅ WeatherAgent completed in 1.40s
2026-01-03 16:43:13,747 - travel_planner - INFO - ✅ Weather node completed
2026-01-03 16:43:13,752 - travel_planner - INFO - 🏨 Starting hotel node
2026-01-03 16:43:13,753 - travel_planner - INFO - 🚀 HotelAgent starting (attempt 1/3)
2026-01-03 16:43:24,292 - travel_planner - INFO - ✅ HotelAgent completed in 10.54s
2026-01-03 16:43:24,294 - travel_planner - INFO - ✅ Hotel node completed
2026-01-03 16:43:24,298 - travel_planner - INFO - 💰 Starting budget node
2026-01-03 16:43:24,300 - travel_planner - INFO - 🚀 BudgetAgent starting (attempt 1/3)
2026-01-03 16:43:32,999 - travel_planner - 


✨ TRIP PLAN COMPLETED SUCCESSFULLY ✨

💰 FINAL BUDGET ESTIMATE: $8,965.00

📅 ITINERARY SUMMARY:
--------------------
Okay, here is a detailed 5-day luxury art-focused itinerary for Rome and Florence, designed for three people, starting September 15, 2024, incorporating all the research and requirements provided.

**Day 1: Ancient Rome & Trastevere Charm** (September 15, 2024)

**Morning (9:00 AM - 12:00 PM)**
- Colosseum & Roman Forum Private Tour
  - What: Explore the Colosseum and Roman Forum with a private guide, focusing on historical context and architectural details.
  - Where: Colosseum, Piazza del Colosseo, 1, 00184 Roma RM, Italy
  - Duration: 3 hours
  - Cost: $250 per person (includes entry fees and private guide)
  - Tips: Book the tour in advance. Wear comfortable shoes. Meet the guide at the designated meeting point near the Colosseum entrance.

**Lunch (12:00 PM - 1:30 PM)**
- Armando al Pantheon
  - Cuisine: Traditional Roman
  - Price: $$$
  - Location: Salita de' Cres